# Quick Start

## Choosing between calibration models and calculating concentrations

In [9]:
from calipytion.tools import Calibrator

In [10]:
# Some measured data of an ABTS standard
conc_unit = "µmol/l"
concentrations = [0.0, 5.0, 10.0, 15.0, 25.0, 50.0, 75.0, 100.0, 150.0, 200.0]
absorption = [
    0.0011,
    0.0790,
    0.1652,
    0.2502,
    0.4306,
    0.9713,
    1.4482,
    1.9507,
    2.8796,
    3.5045,
]

# Create a calibrator object
calibrator = Calibrator(
    molecule_id="https://pubchem.ncbi.nlm.nih.gov/compound/5464076",
    molecule_name="ABTS",
    molecule_symbol="ABTS",
    concentrations=concentrations,
    signals=absorption,
    conc_unit=conc_unit,
)

In [11]:
# add equation for limited growth
lim_model = calibrator.add_model(
    name="limited growth",
    equation="A_max * (1 - exp(-k * ABTS))",
)

In [12]:
calibrator.fit_models()

✅ Models have been fitted successfully.


                                                  Model Overview                                                   
┏━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                ┃     ┃           ┃        ┃                                  ┃ Relative Parameter Standard      ┃
┃ Model Name     ┃ AIC ┃ R squared ┃ RMSD   ┃ Equation                         ┃ Errors                           ┃
┡━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ linear         │ -48 │ 0.9953    │ 0.0817 │ a * ABTS                         │ a: 0.0%,                         │
│ quadratic      │ -44 │ 0.9954    │ 0.0808 │ a * ABTS + b * ABTS**2 + c       │ a: 0.2%, b: 0.0%, c: 5.5%,       │
│ cubic          │ -11 │ 0.8649    │ 0.4358 │ a * ABTS + b * ABTS**2 + c *     │ a: 1.4%, b: 0.0%, c: 0.0%,       │
│                │     │           │        │ ABTS**3                          │                                  │
│ limited growth │ 6   │ 0.1076    │ 1.12   │ A_max * (1 - exp(-k * ABTS))     │ k: n.a., A_max: n.a.,            │
└────────────────┴─────┴───────────┴────────┴──────────────────────────────────┴──────────────────────────────────┘

In [13]:
calibrator.visualize(_static=True)

![svg](ABTS_calibration_curve.svg)

## Calculate concentrations of unknowns

In [14]:
unknowns = [0.12, 1.4, 2, 3, 3.9]

linear_model = calibrator.get_model("linear")
calibrator.calculate_concentrations(linear_model, unknowns, extrapolate=False)

⚠️ Some concentrations could not be calculated and were replaced with nan values, since the provided signal is outside the calibration range. To calculate the concentration outside the calibration range, set 'extrapolate=True'.


array([  6.52078952,  76.07587778, 108.6798254 , 163.0197381 ,
                nan])

In [15]:
standard_report = calibrator.create_standard(
    linear_model, ph=3, temperature=25, temp_unit="C"
)

# save to file
with open("abts_calibration_report.json", "w") as f:
    f.write(standard_report.json())

/Users/max/miniconda3/envs/calibration/lib/python3.11/site-packages/sdRDM/base/datamodel.py:360: UserWarning:

No 'URL' and 'Commit' specified. This model might not be re-usable.

